# Cleaning Data

## Why? Why do we care about cleaning data?

*Discussion: what kinds of problems arise?*

* Missing Data
* Inconsistent Data (e.g. different spellings of the same thing)
* Outliers
* Duplicates

### Where does cleaning belong?

### Techniques

* Regular Expressions
* Data Wrangling with Pandas
* Record Linkage

## Part 1: String Manipulation

A surprising amount of time is spent manipulating strings: cleaning, comparing, extracting information, etc.

 Unfortunately, this isn't a thing computers are particularly efficient at.

Unlike math, where we can parallelize a lot of operations, string manipulation is sequential. It often involves iterating over something one character at a time.

`str.replace()` is a good example of this. It's a very useful function, but it's not very efficient. It has to iterate over the string, and for each character it has to check if it matches the character it's looking for, and if it does, it has to copy the rest of the string over to a new string.

Think a bit about how you'd implement most string functions, and you'll see it's virtually impossible to do anything other than go one character at a time.

This means most string operations are O(n) - they take time proportional to the length of the string.

### String Methods

| Method(s) | Description |
|-----------|-------------|
| `str.upper()`, `str.lower()` | Convert to upper or lower case. |
| `str.isupper()`, `str.islower()` | Check if all characters are upper or lower case. |
| `str.strip()`, `str.lstrip()`, `str.rstrip()` | Remove whitespace from the beginning or end of a string. |
| `str.replace()` | Replace all occurrences of a string with another string. |
| `str.split()` | Split a string into a list of substrings. |
| `str.startswith()`, `str.endswith()` | Check if a string starts or ends with a substring. |
| `substr in str` | Check if a string contains a substring. |
| `str.count()` | Count the number of occurrences of a substring. |
| `str.find()`, `str.rfind()` | Find the index of the first or last occurrence of a substring. |
| `str.index()`,`str.rindex()` | Find the index of the first or last occurrence of a substring. |
| `str.isalpha()`, `str.isalnum()`, `str.isdigit()`, etc. | Check if all characters in a string are alphabetic, alphanumeric, digits, etc. |

<https://docs.python.org/3/library/string.html>

### Common String Manipulation Tasks

* **Searching** - Finding a substring in a string.
* **Replacing** - Replacing a substring with another substring.
* **Splitting** - Splitting a string into a list of substrings based on some character(s).
* **Validating/Matching** - Checking if a string is in a particular format.

## Part 2: Regular Expressions

### Why Regular Expressions?

Whether we're searching or manipulating strings, or a bit of both, it can be very useful to have a way to describe patterns in strings.

Regular expressions provide a common grammar that allows us to describe patterns in strings.
Like SQL, there are many different flavors of regular expressions, but they all have the same basic concepts and more in common than they have different.

### Uses

Regular expressions are used in many different contexts:

* **Searching** - Find all occurrences of a pattern in a string.
* **Validating** - Check if a string matches a pattern. (e.g. phone numbers, email addresses, etc.)
* **Splitting/Extracting** - Extract information from a string based on a pattern.

### Examples

| Pattern | Explanation | Example Matches |
|---------|---------|--|
| `pies?` | Match the word "pie" or "pies" | "pie", "pies" |
| `c[aou]t` | Match words words that start and end with c & t, and have a/o/u in the middle. | "cat", "cot", "cut" |
| `\d{3}-\d{3}-\d{4}` | Match a phone number in dashed format. | "123-456-7890" |
| `[A-Z][a-z]+, [A-Z]{2}` | Match a city name in the format "City, ST" | "Chicago, IL", "Detroit, MI" |
| `\d\s*(\w+)` | Match a number followed by zero or more spaces followed by one or more letters. Capture the letters. | 1 apple -> apple, 2   oranges -> oranges, 3bananas -> bananas |


We'll learn more about these patterns in the next section, first let's take a look at how to use them in Python.


## Part 2: Regular Expressions in Python

Python, like many other languages, has a built-in regular expression module.

* `re.findall` - Find all occurrences of a pattern in a string.
* `re.finditer` - Find all occurrences of a pattern in a string, and return an iterator.
* `re.fullmatch` - Check if a string matches a pattern exactly.
* `re.match` - Check if a string matches a pattern from the start.
* `re.search` - Find the first occurrence of a pattern in a string.
* `re.sub` - Replace all occurrences of a pattern in a string with another string.
* `re.split` - Split a string into a list of substrings based on a pattern.

<https://docs.python.org/3/library/re.html>

### `re.findall` & `re.finditer`

`re.findall` is used to find all occurrences of a pattern in a string and return them all at once in a list.

`re.finditer` returns a lazy iterator of all matches that'll let you iterate over them one at a time.

In [ ]:
import re

# find all four letter words
re.findall(r" (\w{4}) ", "The quick brown fox jumps over the lazy dog.")

### `re.fullmatch`, `re.match`, `re.search`

These three functions all take a pattern and a string, and return a match object if the pattern is found in the string.

* `re.fullmatch` only matches if the pattern matches the entire string.
* `re.match` only matches at the beginning of the string.  (Meaning if the pattern is found at the beginning of the string, but the string continues after that, it still counts as a match.)
* `re.search` matches anywhere in the string.

In [ ]:
import re

def validate_phone_number(phone_number):
    return re.fullmatch(r"\d{3}-\d{3}-\d{4}", phone_number)

def validate_phone_number_bad(phone_number):
    if len(phone_number) != 12:
        return False
    if phone_number[3] != '-':
        return False
    if phone_number[7] != '-':
        return False
    for i in range(12):
        if i == 3 or i == 7:
            continue
        if not phone_number[i].isdigit():
            return False
    return True

In [ ]:
# which will match and which will not?
pattern = r"\d{3}-\d{3}-\d{4}"
functions = [re.fullmatch, re.search, re.match]
strings = ["202-111-5555", "Emily's number is 555-123-4444", "202-111-3300abcdef"]

for f in functions:
    for s in strings:
        # print the name of the function, the string, and the result
        matches = f(pattern, s) is not None
        print(f"{f.__name__:<10} {s:<40} {matches}")

`validate_phone_number` is much easier to read and understand, and it is much easier to maintain.  It also takes less than a second to run 1,000,000 validations.

The naive `validate_phone_number_bad` takes about twice as long. With large data sets, and dozens of complex validations, the difference can be huge.

### `re.sub`

`re.sub` is used to replace all occurrences of a pattern in a string.

In [ ]:
import re

text = "The quick brown fox jumps over the lazy dog."
text = re.sub(r"\s\w{3}\s", "~", text)
print(text)
# "Th~ q~ck br~wn f~x j~mps ~v~r th~ l~zy d~g."

`re.sub` is very fast, and often allows us to do things that are much more complicated than a simple string replace would thus multiplying the speedup.

### `re.split`

`re.split` is used to split a string into a list of substrings based on a pattern. This is less commonly used, but can be useful if something like the standard CSV parser can't handle a particular format.

In [ ]:
# split string apart on punctuation (similar to str.split but can use patterns)
re.split(r"Where", "Wow! Where? I don't know")

### `re.compile`

If you are going to use the same pattern multiple times, it is more efficient to compile the pattern into a regular expression object.

For validation for example:

In [ ]:
import re

phone_number_pattern = re.compile(r"\d{3}-\d{3}-\d{4}")

# phone_number pattern is a compiled regular expression object
print(type(phone_number_pattern))
print(phone_number_pattern)

def validate_phone_number(phone_number):
    # re.Pattern objects have all of the same methods that `re` does,
    # you just omit the pattern argument
    return phone_number_pattern.fullmatch(phone_number)

Cuts the time down to only 0.4s for 1,000,000 validations. A 60% speedup.

All of the methods above can be called on a compiled regular expression object.

(e.g. `re.findall(pattern, text)` is the same as `pattern.findall(text)` if you've compiled the pattern.)

### Flags

Python's regular expression module supports a number of flags that can be passed to the `re.compile` function or any of the methods.

* `re.IGNORECASE` - ignore case when matching
* `re.MULTILINE` - treat the string as multiple lines when evaluating certain patterns (e.g. `^` and `$`)
* `re.DOTALL` - allow `.` to match newlines
* `re.VERBOSE` - allow comments and whitespace in the pattern

In [ ]:
import re

text = """
The quick brown fox jumps over the lazy dog...
Lorem ipsum dolor sit amet, consectetur adipiscing elit...
But I must explain to you how all this mistaken idea...
Then I saw the storm coming...
"""

pattern = re.compile(r"""
    $      # start of line
    \w{3}  # 3 letter word
    \s     # whitespace
    \w*    # any number of letters
    \s     # whitespace
    \w*    # include third word
""", re.VERBOSE | re.IGNORECASE | re.MULTILINE) # combine flags with the | operator

pattern.findall(text) # ["The quick brown", "But I must explain"]

## Part 3: Regular Expression Syntax

Regular expressions describe patterns.

**A character on it's own is a pattern that matches that character.**

This means the regex `a` will match the string `a` but not `b` or `A`.

**`ab` matches `ab` but not `a`, `b`, `ba`, or `AB`.**

This is true for any combination of symbols.  As `a` and `b` grow to be full patterns of their own, this rule will hold.

i.e. `\w{3}[aeiou]?s` and `\s\d{4}` are both valid patterns which we'll learn about shortly, this means they can be combined into
the pattern `\w{3}[aeiou]?s\s\d{4}` which will match `dogs 1234` but not `dog 1234` or `dogs1234`.

While this looks complex now, once we know how to break it into parts we'll see this allows us to read and understand complex patterns.

**`a|b` matches `a` or `b`**

This is called alternation.  It is useful for matching multiple patterns.

**Parenthesis \( \) can be used for grouping**

This means we can write `(ab|cd)` which will match `ab` or `cd`.


### Character Classes

While it'd be possible to write a regex like

(1|2|3|4|5|6|7|8|9|0) to match a single digit, it's much easier to use a range or character class.

[] - matches any character in the brackets, allows lexical ranges
Prefixing the character class with a ^ will match any character not in the brackets.

* `[0-9]` - matches any digit
* `[^abc]` - matches any character except a, b, or c
* `[a-z]` - matches any lowercase letter
* `[A-Z]` - matches any uppercase letter
* `[^ \n\t]` - matches any character except whitespace
* `[a-zA-Z]` - matches any letter in either case
* `[aeiou]` - matches any vowel

What about these?

* `[^aeiou]`
* `[^0-9 ]`
* `[-_a-zA-Z0-9]`
* `[a-mxyz]`

A couple of things to note:

* There are letters (and numbers!) that are not in the range a-z or A-Z from non-english languages. These are not included in the above ranges when specifying a-z.
* If you need to match a literal like `]` that has a special meaning, you prefix it with a backslash.

Certain character classes are so common they have their own shortcuts:

* `\d` - matches any digit
* `\D` - matches any non-digit
* `\w` - matches any alphanumeric character
* `\W` - matches any non-alphanumeric character
* `\s` - matches any whitespace character
* `\S` - matches any non-whitespace character

### Quantifiers

Quantifiers are used to specify how many times a pattern should be matched.

These quantifiers can occur after any pattern:

* `?` - match 0 or 1 times
* `*` - match 0 or more times
* `+` - match 1 or more times
* `{n}` - match exactly n times
* `{n,}` - match at least n times
* `{n,m}` - match at least n times but no more than m times

If there is no quantifier, the pattern is matched exactly once.

* `a?` - matches 0 or 1 `a`
* `\w*` - matches 0 or more alphanumeric characters
* `\s+` - matches 1 or more whitespace characters
* `\d{3}` - matches exactly 3 digits
* `\d{3,}` - matches at least 3 digits
* `\d{3,5}` - matches at least 3 digits but no more than 5

Greediness is the default behavior of quantifiers.  This means they will match as many times as possible.

If you want to make an operator non-greedy, you can add a `?` after it.  This is commonly used with `*` and `+` which can otherwise consume too much of the string.

* `a*?` - matches 0 or more `a` but as few as possible based on the rest of the pattern
* `\W+?` - matches 1 or more non-alphanumeric characters but as few as possible based on the rest of the pattern

### Anchors

Anchors are used to match the beginning or end of a string.

* `^` - matches the beginning of a string
* `$` - matches the end of a string
* `\A` - matches the beginning of a line (same as ^ if in MULTILINE mode)
* `\Z` - matches the end of a line (same as $ if in MULTILINE mode)
* `\b` - matches a word boundary (a special symbol that looks for the boundary between a sequence of alphanumeric characters and a sequence of non-alphanumeric characters)

### Groups

Parentheses can be used to group patterns together.

This is useful for applying quantifiers to multiple patterns at once.  For example:

* `(ab)+` - matches 1 or more `ab`, but not `a` or `b` on their own.
* `(a|an|the)?` - optionally matches `an`, `a`, or `the`

It also allows us to refer to the group later in the pattern.

* `(\w+) \1` - matches a word followed by the same word again.  For example, `the the` or `dog dog` but not `the dog` or `dog the`.

\1 refers to the first group, \2 refers to the second group, etc.

This can also be used when using the `sub` method of the `re` module, for example:

In [ ]:
import re

text = "Hello, and welcome to CAPP 30122, most of you have taken CAPP 30121."
re.sub(r"30(\d{3})", r"99\1", text)
 # "Hello, and welcome to CAPP 99122, most of you have taken CAPP 99121."